In [1]:
import pandas as pd
import numpy as np
import json
import nltk
import re
import csv
import matplotlib.pyplot as plt 
import seaborn as sns
from tqdm import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

In [2]:
action_df = pd.read_csv("/home/yinghui/code/chooj202/movie_genre_prediction/movie_genre_prediction/raw_data/csv/action_df.csv")
adventure_df = pd.read_csv("/home/yinghui/code/chooj202/movie_genre_prediction/movie_genre_prediction/raw_data/csv/adventure_df.csv")
animation_df = pd.read_csv("/home/yinghui/code/chooj202/movie_genre_prediction/movie_genre_prediction/raw_data/csv/animation_df.csv")
biography_df = pd.read_csv("/home/yinghui/code/chooj202/movie_genre_prediction/movie_genre_prediction/raw_data/csv/biography_df.csv")
comedy_df = pd.read_csv("/home/yinghui/code/chooj202/movie_genre_prediction/movie_genre_prediction/raw_data/csv/comedy_df.csv")
crime_df = pd.read_csv("/home/yinghui/code/chooj202/movie_genre_prediction/movie_genre_prediction/raw_data/csv/crime_df.csv")
documentary_df = pd.read_csv("/home/yinghui/code/chooj202/movie_genre_prediction/movie_genre_prediction/raw_data/csv/documentary_df.csv")

merged_df = pd.concat([action_df, adventure_df, animation_df, biography_df, comedy_df, crime_df, documentary_df])
merged_df.shape

(1500, 6)

In [3]:
merged_df.sample(5)

,Unnamed: 0,movie,imdb_id,genre,plot,image_url
175,175,The Peanuts Movie,tt2452042,"['Animation', ' Adventure', ' Comedy']","Charlie Brown, Snoopy, and the whole gang are ...",http://img.omdbapi.com/?i=tt2452042&h=600&apik...
64,64,Captain Phillips,tt1535109,"['Action', ' Biography', ' Crime']",Captain Phillips is a multi-layered examinatio...,http://img.omdbapi.com/?i=tt1535109&h=600&apik...
144,144,Blue Velvet,tt0090756,"['Crime', ' Drama', ' Mystery']",College student Jeffrey Beaumont returns to hi...,http://img.omdbapi.com/?i=tt0090756&h=600&apik...
136,136,Interstella 5555: The 5tory of the 5ecret 5tar...,tt0368667,"['Animation', ' Adventure', ' Music']",An unscrupulous music executive and his flunky...,http://img.omdbapi.com/?i=tt0368667&h=600&apik...
238,238,Monster House,tt0385880,"['Animation', ' Comedy', ' Family']",13-year-old DJ is observing his neighbor Nebbe...,http://img.omdbapi.com/?i=tt0385880&h=600&apik...


In [4]:
merged_df = merged_df.drop(columns=['Unnamed: 0'])

In [5]:
merged_df = merged_df.drop_duplicates()
merged_df.shape

(1094, 5)

In [6]:
merged_df.sample(5)

,movie,imdb_id,genre,plot,image_url
188,Snow White and the Seven Dwarfs,tt0029583,"['Animation', ' Adventure', ' Family']",The beautiful and kindhearted princess Snow Wh...,http://img.omdbapi.com/?i=tt0029583&h=600&apik...
161,Fantasia 2000,tt0120910,"['Animation', ' Family', ' Fantasy']",In this update of Disney's masterpiece film mi...,http://img.omdbapi.com/?i=tt0120910&h=600&apik...
75,Stalag 17,tt0046359,"['Comedy', ' Drama', ' War']",It's a dreary Christmas 1944 for the American ...,http://img.omdbapi.com/?i=tt0046359&h=600&apik...
231,Dead Man,tt0112817,"['Adventure', ' Drama', ' Fantasy']","In the late-1800s, the meek accountant from Cl...",http://img.omdbapi.com/?i=tt0112817&h=600&apik...
69,Straight Outta Compton,tt1398426,"['Biography', ' Drama', ' History']","In 1987, five young men, using brutally honest...",http://img.omdbapi.com/?i=tt1398426&h=600&apik...


In [19]:
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords 
from nltk.stem import WordNetLemmatizer

In [17]:
def cleaning(sentence):
    
    # Basic cleaning
    sentence = sentence.strip() ## remove whitespaces
    sentence = sentence.lower() ## lowercase 
    sentence = ''.join(char for char in sentence if not char.isdigit()) ## remove numbers
    
    # Advanced cleaning
    for punctuation in string.punctuation:
        sentence = sentence.replace(punctuation, '') ## remove punctuation
    
    tokenized_sentence = word_tokenize(sentence) ## tokenize 
    stop_words = set(stopwords.words('english')) ## define stopwords
    
    tokenized_sentence_cleaned = [ ## remove stopwords
        w for w in tokenized_sentence if not w in stop_words
    ]

    lemmatized = [
        WordNetLemmatizer().lemmatize(word, pos = "v") 
        for word in tokenized_sentence_cleaned
    ]
    
    cleaned_sentence = ' '.join(word for word in lemmatized)
    
    return cleaned_sentence

In [20]:
merged_df["clean_plot"] = merged_df["plot"].apply(cleaning)

In [21]:
merged_df

,movie,imdb_id,genre,plot,image_url,clean_plot
0,Ramayana: The Legend of Prince Rama,tt0259534,"['Animation', ' Action', ' Adventure']","The fantastic story of Rama, a young prince wh...",http://img.omdbapi.com/?i=tt0259534&h=600&apik...,fantastic story rama young prince banish fores...
1,The Dark Knight,tt0468569,"['Action', ' Crime', ' Drama']",Set within a year after the events of Batman B...,http://img.omdbapi.com/?i=tt0468569&h=600&apik...,set within year events batman begin batman lie...
2,The Lord of the Rings: The Return of the King,tt0167260,"['Action', ' Adventure', ' Drama']",The final confrontation between the forces of ...,http://img.omdbapi.com/?i=tt0167260&h=600&apik...,final confrontation force good evil fight cont...
3,Spider-Man: Across the Spider-Verse,tt9362722,"['Animation', ' Action', ' Adventure']",Miles Morales returns for the next chapter of ...,http://img.omdbapi.com/?i=tt9362722&h=600&apik...,miles morales return next chapter oscar®winnin...
4,The Message,tt0075143,"['Action', ' Adventure', ' Biography']","The story of prophet ""Muhammad"" and the delive...",http://img.omdbapi.com/?i=tt0075143&h=600&apik...,story prophet muhammad delivery message god allah
...,...,...,...,...,...,...
236,The Town,tt0840361,"['Crime', ' Drama', ' Thriller']",The Charlestown neighborhood of Boston is reno...,http://img.omdbapi.com/?i=tt0840361&h=600&apik...,charlestown neighborhood boston renowned churn...
238,Kiss Kiss Bang Bang,tt0373469,"['Comedy', ' Crime', ' Mystery']",A petty thief posing as an actor is brought to...,http://img.omdbapi.com/?i=tt0373469&h=600&apik...,petty thief pose actor bring los angeles unlik...
242,Lolita,tt0056193,"['Crime', ' Drama', ' Romance']",Humbert Humbert forces a confrontation with a ...,http://img.omdbapi.com/?i=tt0056193&h=600&apik...,humbert humbert force confrontation man whose ...
243,Perfume: The Story of a Murderer,tt0396171,"['Crime', ' Drama', ' Fantasy']",Jean-Baptiste Grenouille (Ben Whishaw) came in...,http://img.omdbapi.com/?i=tt0396171&h=600&apik...,jeanbaptiste grenouille ben whishaw come world...


In [22]:
merged_df["genre"] = (
    merged_df["genre"]
    .apply(eval)
    .apply(lambda x: [genre.strip() for genre in x])
    )

In [23]:
#check the genre we have
merged_df["genre"].explode().value_counts()

Drama        641
Adventure    395
Comedy       383
Action       292
Crime        271
Animation    250
Biography    250
Thriller      90
Romance       85
History       72
Family        67
Mystery       56
Fantasy       54
Sci-Fi        53
Music         27
War           27
Film-Noir     25
Sport         21
Musical       12
Horror        11
Western        9
Name: genre, dtype: int64

In [24]:
from sklearn.preprocessing import MultiLabelBinarizer

multilabel_binarizer = MultiLabelBinarizer()
multilabel_binarizer.fit(merged_df["genre"])

# transform target variable
y = multilabel_binarizer.transform(merged_df['genre'])

In [25]:
y.shape

(1094, 21)

In [26]:
genre_names = multilabel_binarizer.classes_

# Adding 
for i in range(len(genre_names)):
    merged_df[f"{genre_names[i]}"] = y[:,i]

merged_df.shape

(1094, 27)

In [27]:
merged_df.head(5)

,movie,imdb_id,genre,plot,image_url,clean_plot,Action,Adventure,Animation,Biography,...,Horror,Music,Musical,Mystery,Romance,Sci-Fi,Sport,Thriller,War,Western
0,Ramayana: The Legend of Prince Rama,tt0259534,"[Animation, Action, Adventure]","The fantastic story of Rama, a young prince wh...",http://img.omdbapi.com/?i=tt0259534&h=600&apik...,fantastic story rama young prince banish fores...,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
1,The Dark Knight,tt0468569,"[Action, Crime, Drama]",Set within a year after the events of Batman B...,http://img.omdbapi.com/?i=tt0468569&h=600&apik...,set within year events batman begin batman lie...,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,The Lord of the Rings: The Return of the King,tt0167260,"[Action, Adventure, Drama]",The final confrontation between the forces of ...,http://img.omdbapi.com/?i=tt0167260&h=600&apik...,final confrontation force good evil fight cont...,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Spider-Man: Across the Spider-Verse,tt9362722,"[Animation, Action, Adventure]",Miles Morales returns for the next chapter of ...,http://img.omdbapi.com/?i=tt9362722&h=600&apik...,miles morales return next chapter oscar®winnin...,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
4,The Message,tt0075143,"[Action, Adventure, Biography]","The story of prophet ""Muhammad"" and the delive...",http://img.omdbapi.com/?i=tt0075143&h=600&apik...,story prophet muhammad delivery message god allah,1,1,0,1,...,0,0,0,0,0,0,0,0,0,0


In [31]:
X_train, X_test, y_train, y_test = train_test_split(merged_df['clean_plot'], y, test_size=0.3)

In [32]:
# Instantiating the TfidfVectorizer
tf_idf_vectorizer = TfidfVectorizer(max_df=0.8, max_features=10000)

# Training it on the texts
X_train_vec = pd.DataFrame(tf_idf_vectorizer.fit_transform(X_train).toarray(),
                 columns = tf_idf_vectorizer.get_feature_names_out())
X_test_vec =  pd.DataFrame(tf_idf_vectorizer.transform(X_test).toarray(),
                 columns = tf_idf_vectorizer.get_feature_names_out())

In [34]:
X_train_vec.head()

,aamir,aaron,abagnale,abandon,abandonment,abate,abbys,abduct,abilities,ability,...,zoe,zombie,zombiekiller,zombies,zone,zoo,zoology,zootopia,zorg,zorro
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [35]:
X_test_vec.head()

,aamir,aaron,abagnale,abandon,abandonment,abate,abbys,abduct,abilities,ability,...,zoe,zombie,zombiekiller,zombies,zone,zoo,zoology,zootopia,zorg,zorro
0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.125182,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
